# Gaussian Naive Bayes Classifier from Scratch

In [1]:
import numpy as np
from random import randrange
import csv
import math

def load_csv_dataset(filename):
    lines = csv.reader(open(filename, 'r'))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]  
        # Convert String to Float numbers
    return dataset

def mean(numbers):
    return np.mean(numbers)


def stdev(numbers):
    return np.std(numbers)

def cross_validation_split(dataset, n_folds):
    #Split dataset into the k folds. Returns the list of k folds
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


def evaluate_algorithm(dataset, algorithm, n_folds, ):
    #Evaluate an algorithm using a cross validation split
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, )
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores


# Naive Bayes Classification

In [2]:
def separate_by_class(dataset):
    #Split training set by class value
    separated = {}
    for i in range(len(dataset)):
        row = dataset[i]
        if row[-1] not in separated:
            separated[row[-1]] = []
        separated[row[-1]].append(row)
    return separated


def model(dataset):
    #Find the mean and standard deviation of each feature in dataset
    models = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    models.pop() 
    #Remove last entry because it is class value.
    return models


def model_by_class(dataset):
    #The same Model function, but for each feature in dataset by their class
    separated = separate_by_class(dataset)
    class_models = {}
    for (classValue, instances) in separated.items():
        class_models[classValue] = model(instances)
    return class_models

def calculate_pdf(x, mean, stdev):
    #Probability using gaussian density function
    if stdev == 0.0:
        if x == mean:
            return 1.0
        else:
            return 0.0
    exponent = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return 1 / (math.sqrt(2 * math.pi) * stdev) * exponent


def calculate_class_probabilities(models, input):
    #Calculate the class probability for input sample and combine them for each feature
    probabilities = {}
    for (classValue, classModels) in models.items():
        probabilities[classValue] = 1
        for i in range(len(classModels)):
            (mean, stdev) = classModels[i]
            x = input[i]
            probabilities[classValue] *= calculate_pdf(x, mean, stdev)
    return probabilities

def predict(models, inputVector):
    #Compare and return the class label which has max probability
    probabilities = calculate_class_probabilities(models, inputVector)
    (bestLabel, bestProb) = (None, -1)
    for (classValue, probability) in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel


def getPredictions(models, testSet):
    #To get class label for each value in test set
    predictions = []
    for i in range(len(testSet)):
        result = predict(models, testSet[i])
        predictions.append(result)
    return predictions


def naive_bayes(train, test, ):
    #Creating a Naive Bayes Model
    summaries = model_by_class(train)
    predictions = getPredictions(summaries, test)
    return predictions

# The Main Function

In [3]:
def main():
    # load and prepare data
    filename = 'banknote.csv'
    dataset = load_csv_dataset(filename)

    n_folds = 3
    
    print("S6 B Batch AI Assignment")
    print("Group 8 Ritu, Muhsin, Aakash, Rahul Chaudhary, Rahul Anand")
    print("Gaussian Naive Bayes from Scratch")
    accuracy_naive = evaluate_algorithm(dataset, naive_bayes, n_folds)
    print("Naive Bayes Classification")
    print('Accuracy in each fold: %s' % accuracy_naive)
    print ('Average Accuracy: %f' % (sum(accuracy_naive) / len(accuracy_naive)))

"""
Bank note authentication dataset
http://archive.ics.uci.edu/ml/datasets/banknote+authentication
Reference: (https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/)
"""
main()

S6 B Batch AI Assignment
Group 8 Ritu, Muhsin, Aakash, Rahul Chaudhary, Rahul Anand
Gaussian Naive Bayes from Scratch
Naive Bayes Classification
Accuracy in each fold: [82.4945295404814, 85.12035010940919, 85.99562363238512]
Average Accuracy: 84.536834
